# Introduction
Query auto completion (QAC) is an important component of writing assitance in search engines. Given an incomplete user input, QAC suggests a list of possible completions that help save users' keystrokes. These completions usually start with the user input. For example, when users type in "linke", QAC may suggests "linkedin", "linkedin corporation", etc..

A typical QAC system usually consists of two main components: candidate generation and candidate ranking. Given a user input, the candidate generation component generates a list of candidate suggestions first, and the candidate ranking component ranks the suggestions in the order of relevance. 

In this session, we will go through the steps of building a QAC system. The content covers


1.   sample dataset preparation
2.   candidate generation component construction
3.   candidate ranking component construction
4.   a QAC system in action



In [ ]:
# Download a sample data of the AOL (America Online) dataset from the web
# The complete collection consists of ~20M web queries collected from ~650k AOL users over three months (2006 March to 2006 June).
# !wget http://www.cim.mcgill.ca/~dudek/206/Logs/AOL-user-ct-collection/user-ct-test-collection-01.txt

In [ ]:
# Take a glimpse into the data. We will only make use of the "Query" and the "QueryTime" column
# !head user-ct-test-collection-01.txt

In [ ]:
# !wc -l user-ct-test-collection-01.txt

In [1]:
# Extract query and split background/train/dev/test data based on QueryTime
import string, re, random
import tensorflow
random.seed(1234)  # control repeatability

sample_ratio = 0.01

punctuation = string.punctuation
translator = re.compile('[{}]+'.format(re.escape(string.punctuation)))  # regex to identify consecutive punctuations


# Candidate Generation Component Construction
In this section, we will use the background queries to construct a finite state transducer (FST) based candidate generator. The process is similar to a trie look up.

Given a query "abc", the FST will look up top k most frequent background queries that starts with "abc".

In [2]:
# Install an open-source autocompletion package that supports trie-based completion (https://zepworks.com/posts/you-autocomplete-me/)
# You can also use ElasticSearch for the same purpose (https://www.elastic.co/guide/en/elasticsearch/reference/current/search-suggesters.html#completion-suggester)
# !pip install fast-autocomplete

In [3]:
import pandas as pd
import numpy as np
df=pd.read_csv("../all-data2.csv",usecols=['Text'])

In [31]:
import re
def clean(text):
    text=str(text)
    text=re.sub(r"[`:,/!$%^&*|+'\\]","",text.lower())
    text=text.strip().replace("  "," ")
    return text
df['Text']=df['Text'].apply(func=clean)
paragraph = df['Text'].values

In [30]:
df.head()

,Text
0,technopolis plans to develop in stages an area...
1,the international electronic industry company ...
2,with the new production plant the company woul...
3,according to the company s updated strategy fo...
4,financing of aspocomp s growth aspocomp is agg...


In [21]:
def makeQuery(texts,indexes):
    for text,indx in zip(texts,indexes):
        words=text.split()
        for i,w in enumerate(words):
            yield " ".join(words[i:])+" "+str(indx)

In [12]:
bkg_queries=df.Text[:int(len(df)*1)]
# train_queries=df.Text[int(len(df)*0.8):int(len(df)*0.85)]
# dev_queries=df.Text[int(len(df)*0.85):int(len(df)*0.9)]
# test_queries=df.Text[int(len(df)*0.9):]

In [14]:
# query_log

In [22]:
# Construct the query logs using background queries
# Write down the logs in a format accepted by the package
from collections import Counter
bkg_query_count = Counter(list(makeQuery(bkg_queries,bkg_queries.index)))

query_log = dict()
for query, count in bkg_query_count.items():
  query_log[query] = [dict(), query, count]

import json
with open('query_log.json', 'w') as outfile:
    json.dump(query_log, outfile, indent=2)

In [23]:
!head -n 10 query_log.json

{
  "Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . 0": [
    {},
    "Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . 0",
    1
  ],
  "plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . 0": [
    {},
    "plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . 0",
    1


In [24]:
from fast_autocomplete import autocomplete_factory

content_files = {
    'words': {
        'filepath': 'query_log.json',
        'compress': True  # means compress the graph data in memory
    }
}

candidate_generator = autocomplete_factory(content_files=content_files)

In [51]:
indexes=[]
for s in inputDict.values():
    results=candidate_generator.search(s, max_cost=0, size=50)
    indexes.extend([ int(r[0].split()[-1]) for r in results])

In [52]:
indexes

[42, 4549, 9, 20, 66, 66, 42, 4549, 9, 20, 66, 42, 20, 66]

In [35]:
def find(text,inputDict):
    score={'Name':1,"SurName":1,"qulification":5,"Mobile":5,"Email":5,"Work":5}
    scored=0
    for key in inputDict.keys():
        textlst=text.lower().split()
        n=textlst.count(inputDict[key])
        scored+=n*score[key]
    return scored

In [81]:
inputDict={'Name':"shivam","SurName":"sharma","qulification":"m.tech",
           "Mobile":"9530450649","Email":"","Work":""}

In [85]:
def predict(inputDict):
    indexes=[]
    for s in inputDict.values():
        results=candidate_generator.search(s, max_cost=0, size=50)
        indexes.extend([ int(r[0].split()[-1]) for r in results])
    output_data = pd.DataFrame(columns=["index","text","score"])
    count=0
    for index in set(indexes):
        text=paragraph[index]
        score=find(text,inputDict)
        output_data.loc[count]=([index,text,score])
        count+=1
    return output_data.sort_values('score',axis=0,ascending=False)

In [86]:
res=predict(inputDict)

In [87]:
res

,index,text,score
0,66,stock exchange announcement 20 july 2006 1 ( 1...,1
1,4549,my name is sourabhh sharma i study in jecrc ja...,1
2,9,teliasonera tlsn said the offer is in line wit...,1
3,42,sales for both the department store division a...,1
4,20,( filippova ) a trilateral agreement on invest...,1
5,4251,the e7 smartphone will be available for rs35 0...,0
